In [1]:
from pyspark import SparkContext, SparkConf, SQLContext

In [19]:
conf = SparkConf().set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.11:2.3.2").setAppName("netflix").setMaster("local[1]")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=netflix, master=local[1]) created by __init__ at /tmp/ipykernel_65/776453987.py:2 

In [5]:
sqlContext = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
df = sqlContext.read.csv("data.csv", header=True)

In [8]:
df.show(5)

+---+----------+----------+----------+--------------------+------+---------------+-------------+----------+----+
| id|      date|first_name| last_name|               email|gender|     ip_address|      country|price_paid| tax|
+---+----------+----------+----------+--------------------+------+---------------+-------------+----------+----+
|  1|03/28/2021|   Marylin|    Alders|malders0@shop-pro.jp|  Male|  160.119.8.119|       Canada|     $5.36|null|
|  2|03/19/2021|     Hinda|  Harridge|  hharridge1@gnu.org|Female| 210.51.172.132|       Canada|     $8.32|null|
|  3|03/17/2021|       Erl|Kilminster|ekilminster2@etsy...|  Male|  213.8.101.145|United States|     $3.73|  20|
|  4|03/19/2021|     Grata|    Brantl|    gbrantl3@umn.edu|Female|  5.140.237.101|United States|     $3.70|  20|
|  5|03/24/2021|     Kanya|   Beasant| kbeasant4@jigsy.com|  Male|106.252.162.233|      Morocco|     $9.24|  20|
+---+----------+----------+----------+--------------------+------+---------------+-------------+

In [9]:
df = df.withColumn("price_paid", df.price_paid.substr(2,4))

In [10]:
df.show(5)

+---+----------+----------+----------+--------------------+------+---------------+-------------+----------+----+
| id|      date|first_name| last_name|               email|gender|     ip_address|      country|price_paid| tax|
+---+----------+----------+----------+--------------------+------+---------------+-------------+----------+----+
|  1|03/28/2021|   Marylin|    Alders|malders0@shop-pro.jp|  Male|  160.119.8.119|       Canada|      5.36|null|
|  2|03/19/2021|     Hinda|  Harridge|  hharridge1@gnu.org|Female| 210.51.172.132|       Canada|      8.32|null|
|  3|03/17/2021|       Erl|Kilminster|ekilminster2@etsy...|  Male|  213.8.101.145|United States|      3.73|  20|
|  4|03/19/2021|     Grata|    Brantl|    gbrantl3@umn.edu|Female|  5.140.237.101|United States|      3.70|  20|
|  5|03/24/2021|     Kanya|   Beasant| kbeasant4@jigsy.com|  Male|106.252.162.233|      Morocco|      9.24|  20|
+---+----------+----------+----------+--------------------+------+---------------+-------------+

In [11]:
df = df.withColumn("price_paid",df.price_paid.cast('double'))

In [12]:
df.show(5)

+---+----------+----------+----------+--------------------+------+---------------+-------------+----------+----+
| id|      date|first_name| last_name|               email|gender|     ip_address|      country|price_paid| tax|
+---+----------+----------+----------+--------------------+------+---------------+-------------+----------+----+
|  1|03/28/2021|   Marylin|    Alders|malders0@shop-pro.jp|  Male|  160.119.8.119|       Canada|      5.36|null|
|  2|03/19/2021|     Hinda|  Harridge|  hharridge1@gnu.org|Female| 210.51.172.132|       Canada|      8.32|null|
|  3|03/17/2021|       Erl|Kilminster|ekilminster2@etsy...|  Male|  213.8.101.145|United States|      3.73|  20|
|  4|03/19/2021|     Grata|    Brantl|    gbrantl3@umn.edu|Female|  5.140.237.101|United States|       3.7|  20|
|  5|03/24/2021|     Kanya|   Beasant| kbeasant4@jigsy.com|  Male|106.252.162.233|      Morocco|      9.24|  20|
+---+----------+----------+----------+--------------------+------+---------------+-------------+

In [13]:
df.groupBy("country").sum("price_paid").collect()

[Row(country='France', sum(price_paid)=2619.61),
 Row(country='United States', sum(price_paid)=2183.6700000000005),
 Row(country='Morocco', sum(price_paid)=362.96999999999997),
 Row(country='Canada', sum(price_paid)=1017.4500000000005),
 Row(country=None, sum(price_paid)=278.22999999999996)]

In [71]:
df2 = df.groupBy("country","gender").sum("price_paid")

In [72]:
df2.show()

+-------------+------+------------------+
|      country|gender|   sum(price_paid)|
+-------------+------+------------------+
|       France|  Male|           1176.31|
|       Canada|Female|            475.03|
|       France|Female|1263.7199999999998|
|      Morocco|Female|189.67000000000002|
|      Morocco|  Male|141.38000000000002|
|United States|  Male| 960.8399999999997|
|United States|Female|1094.3100000000009|
|       Canada|  Male| 458.8100000000001|
|       France|  null|179.58000000000004|
|         null|  null|             39.43|
|United States|  null|128.51999999999998|
|         null|Female|142.53000000000003|
|      Morocco|  null|             31.92|
|       Canada|  null| 83.61000000000001|
|         null|  Male| 96.27000000000001|
+-------------+------+------------------+



In [73]:
! pip install pymongo

In [74]:
import pandas
import json

df2 = df2.toPandas()
df2.to_json("test.json")
with open("test.json") as f:
    file_data = json.load(f)

In [75]:
import pymongo
myclient = pymongo.MongoClient("mongodb://849599b81ae5:27017/")
mydb = myclient["netflix"]
mycol = mydb["stats"]

In [76]:
print(myclient.list_database_names())

['admin', 'config', 'local', 'netflix']


In [79]:
x = mycol.insert_one(file_data)
print(x)

DuplicateKeyError: E11000 duplicate key error collection: netflix.stats index: _id_ dup key: { _id: ObjectId('619feee530c95f1215930768') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('619feee530c95f1215930768')}, 'errmsg': "E11000 duplicate key error collection: netflix.stats index: _id_ dup key: { _id: ObjectId('619feee530c95f1215930768') }"}

In [80]:
print(x)
client.close()

In [82]:
result = mycol.find()

In [93]:
print(result[0]['country'])

{'0': 'France', '1': 'Canada', '2': 'France', '3': 'Morocco', '4': 'Morocco', '5': 'United States', '6': 'United States', '7': 'Canada', '8': 'France', '9': None, '10': 'United States', '11': None, '12': 'Morocco', '13': 'Canada', '14': None}


In [95]:
print(result[0]['sum(price_paid)'])

{'0': 1176.31, '1': 475.03, '2': 1263.72, '3': 189.67, '4': 141.38, '5': 960.84, '6': 1094.31, '7': 458.81, '8': 179.58, '9': 39.43, '10': 128.52, '11': 142.53, '12': 31.92, '13': 83.61, '14': 96.27}
